Time Series Analysis - Consumer Complaints
================
Andre Nascimento, Senior Software Engineer ([email](mailto:andre.nascimento@teradata.com))

-   [Introduction](#introduction)
-   [Experience](#experience)
    -   [Quick Start](#quick-start)
    -   [Walkthrough](#walkthrough)
        -   [Step 1: Setup](#step-1)
        -   [Step 2: Establishing the Vantage Connection](#step-2)
        -   [Step 3: Querying the Data](#step-3)
        -   [Step 4: Visualizing the Data](#step-4)
        -   [Step 5: Extracting Insights from the Data](#step-5)
-   [Dataset](#dataset)
-   [Explore](#explore)

<a id='introduction'></a>
# Introduction

How can we use **Vantage** to extract insights and tell a story behind a dataset? In this workflow you will see how powerful and simple the **Vantage Jupyter Extensions** are to use to extract answers from a public dataset available through Data.gov. We will analyze the number of complaints over time, using SQL in a Jupyter notebook powered by **Vantage** to answer the following questions:

*What are the trends of complaints over time?*
*How can we interpret the outliers in the dataset?*

By answering questions like the ones above, we will gain a deeper understanding of the dataset and we'll be able to explain in plain English how the number of complaints evolve over time. In the **Explore** section, we will focus on analyzing the number of complaints over time and identifying trends and outliers in the time series to answer the questions above. All of this will be executed in a Teradata SQL JupyterLab environment that is fueled by **Vantage** and allows us even to plot charts and visualize the query results right on the notebook.

For more information on using the **Teradata SQL Extension for Jupyter**, consult the documentation at https://teradata.github.io/jupyterextensions/. This documentation page is only available outside of the workspace.

<a id='experience'></a>
# Experience

<a id='quick-start'></a>
# Quick Start

The **Experience** section takes about 3 minutes to run.

1. Open Jupyter from the Start menu of your Workspace VM.

2. In the sidebar, navigate to **Documents > `consumer-complaints` > `sql-demo.ipynb`**.

> If you are unable to locate the file, see how to [download files from GitLab](https://innersource.teradata.com/published/documentation/blob/master/docs/faq.md#4-how-do-i-download-files-from-gitlab).

3. In **Step 1** of the **Experience** section, enter your Quicklook ID as the `user` parameter of the `%addconnect` command.

![](https://innersource.teradata.com/published/consumer-complaints/raw/master/images/addconnect-user.png)

4. Go to the **Run** tab and click **Run All Cells**.

![](https://innersource.teradata.com/published/consumer-complaints/raw/master/images/jupyterlab-run-all.png)

<a id='walkthrough'></a>
## Walkthrough

<a id='step-1'></a>
#### Step 1: Setup

Let's start by adding a connection to the Teradata database that contains our data. We will use the **Teradata SQL Extension for Jupyter** to invoke a the magic command `%addconnect`.

##### 🚨 Enter your Quicklook ID as the `user` parameter below to add the connection.

In [ ]:
%addconnect name=dev_environment, user=, host=devplatform-vantagecop1.td.teradata.com, PROPS="logmech=LDAP"

<a id='step-2'></a>
#### Step 2: Establishing the Vantage Connection
Now that we have added the connection, let's connect to the database. The password is your Quicklook password.

In [ ]:
%connect NAME=dev_environment

If you have issues connecting to the database, refer to the [FAQ](https://innersource.teradata.com/published/documentation/blob/master/docs/faq.md#44-im-having-trouble-connecting-to-the-vantage-system-using-the-teradata-sql-extension-for-jupyter-what-do-i-do).

<a id='step-3'></a>
#### Step 3: Querying the Data

We will start off by counting the number of rows in the table.

In [ ]:
select count(*) from FinancialProtection_V.consumer_complaints;

As we can see, there are just under 1.3 million rows. That is a lot of data, and to show the power of **Vantage**, we will query all the elements in the table at once.

In [ ]:
select * from FinancialProtection_V.consumer_complaints;

The **Teradata SQL Extension for Jupyter** makes it easy and fast to visualize your query results. We can see from the output above that the query results were paginated in order to make the loading faster on the notebook. This makes an overall great user experience as we don't need to see the full 1.3 million rows at once.

<a id='step-4'></a>
#### Step 4: Visualizing the Data

From the query above, we notice that this dataset has a lot of information. In order to derive some insights, we need to start grouping the data. The very first column is `date_received`. That is the date the complaints were received and it means that we can look at a time series plot of the data. Let's then start by grouping the counts of `complaint_id` over time, using `date_received` as our time axis.

In [ ]:
select date_received, count(complaint_id) as counts
from FinancialProtection_V.consumer_complaints
group by date_received;

This is great; we now have the number of complaints (`counts`) by time (`date_received`), but how do we make sense of this data? Well, the **Teradata SQL Extension for Jupyter** has a magic command for charts, which is just what we need. So let's go ahead and plot this time series on a graph by using the `%chart` magic command. 

In [ ]:
%chart x=date_received, y=counts, title=Number of Complaints over Time, mark=line

By visualizing the data above, we can see that the number of complaints varies a lot over time and there also seem to be more complaints over time progresses. There are also some unusual spikes in 2017. Let's understand more about our data. We'll start by looking at the general trend.

Let's begin by grouping the data by month and replotting the graph above.

In [ ]:
select extract(year from date_received) || extract(month from date_received) as month_date, count(complaint_id) as counts
from FinancialProtection_V.consumer_complaints
group by month_date
order by month_date;

In [ ]:
%chart x=month_date, y=counts, title=Number of Complaints by Month and Year, mark=line

Looking at complaints over month and year, we can see there is clearly an upward trend. One hypothesis is that as time progresses, people get more conscious and spread the word. The media can also advertise the complaint channels over time. Through this chart we can see clearly the spikes that we saw above were in January 2017 and September 2017. Let's dive deeper into these dates and draw some insights on the next step.

<a id='step-5'></a>
#### Step 5: Extracting Insights from the Data

Let's narrow down the two spikes above and see where exactly the are happening. We can do this by ploting another time series plot, this time only in 2017.

In [ ]:
select date_received, count(complaint_id) as counts
from FinancialProtection_V.consumer_complaints
where year(date_received) = 2017
group by date_received
order by date_received;

In [ ]:
%chart x=date_received, y=counts, mark=line, title=Complaints over time - 2017

As we look at the peaks, we find that these happened between January 15<sup>th</sup> - 21<sup>st</sup> and on the first week of Semptember, respectively. Let's find the actual dates of the peaks. We can limit the query to pick up at least 1,500 complaints a day.

In [ ]:
select date_received, 
    month(date_received) as month_date, 
    count(complaint_id) as counts
from FinancialProtection_V.consumer_complaints
where year(date_received) = 2017 and month_date in (1, 9) 
group by date_received
having counts >= 1500
order by month_date, counts desc;

Let us now look at some of the issues that were reported during these dates.

In [ ]:
select date_received, company, count(company) as counts
from FinancialProtection_V.consumer_complaints
where date_received in (
    date '2017-01-19', 
    date '2017-01-20',
    date '2017-09-08',
    date '2017-09-09',
    date '2017-09-13'
)
group by date_received, company
having counts > 500
order by date_received, counts desc;

Interestingly, we can see that the great majority of the the complaints were directed at two companies: Navient Solutions and EQUIFAX. These seem to be highly correlated with the Navient Lawsuit and the Equifax breach events that happened around those dates, respectively. Let's recap what happened on those two events:

> **Navient Lawsuit**:
*On January 2017, the U.S. Consumer Financial Protection Bureau (CFPB) and the Illinois and Washington attorneys general sued Navient Solutions. Navient is a major servicer of private and federal student loans. Accoriding to the CFPB at least since January 2010 "Navient has misallocated payments, steered struggling borrowers toward multiple forbearances instead of income-driven repayment plans, and provided unclear information about how to re-enroll in income-driven repayment plans and how to qualify for a co-signer release"*

> **Equifax Breach:**
*On September 7<sup>th</sup> 2017, Equifax announced a cybersecurity breach, one of the largest in history, had happened from mid-May through July 2017.
Some of the personal information that was accessed included names, social security numbers, birth dates, addresses and driver's license numbers.*

Let's now look at the top issues for Navient Solutions and Equifax during those periods to confirm our hypothesis.

In [ ]:
-- analyze top issues reported agains Navient Soultions on 2017-01-19 and 2017-01-20
select company, product, issue, count(issue) as counts
from FinancialProtection_V.consumer_complaints
where date_received in (
    date '2017-01-19', 
    date '2017-01-20') and
    company like 'Navient Solutions%'
group by company, product, issue
order by counts desc;

We can see the top two issues represent the majority of complaint counts agains Navient Solutions. Furthermore, by looking at the product and issue columns we can infer that they are indeed related to the lawsuit regarding student loans. Now let's do the same analysis for the Equifax issues.

In [ ]:
-- analyze top issues reported agains Navient Soultions on 2017-01-19 and 2017-01-20
select 
    company, 
    product, 
    issue, 
    count(issue) as counts
from FinancialProtection_V.consumer_complaints
where date_received in (
    date '2017-09-08', 
    date '2017-09-09',
    date '2017-09-13') and
        company like 'EQUIFAX%'
group by company, product, issue
order by counts desc;

Here we can also confirm our hypothesis. The top issues talk about improper use of the credit report, fraud alerts, identity theft etc. This really does seem related to the Equifax breach that happened around the same time frame.

<a id='dataset'></a>
# Dataset

The Consumer Complaints Database has complaints data that was received by the Consumer Financial Protection Bureau (CFPB) on financial products and services, which include but are not limited to bank accounts, credit cards, credit reporting, debt collection, money transfers, mortgages, student loans and other types of consumer credit. The dataset is refreshed daily and contains information on the provider, the complaint, date, ZIP code and more. More information about the dataset can be found in the *Consumer* section of the Data.gov website.

The `FinancialProtection_V.consumer_complaints` dataset has 1,273,782 rows, each representing a unique consumer complaint, and 18 columns, representing the following features:

* `date_received`: date that CFPB received the complaint
* `product`: type of product the consumer identified in the complaint
* `sub_product`: type of sub-product the consumer identified in the complaint
* `issue`: issue the consumer identified in the complaint
* `sub_issue`: sub-issue the consumer identified in the complaint
* `consumer_complaint_narrative`: consumer-submitted description of "what happened" from the complaint
* `company_public_response`: company's optional, public-facing response to a consumer's complaint
* `company`: complaint is about this company
* `state`: state of the mailing address provided by the consumer
* `zip_code`: mailing ZIP code provided by the consumer
* `tags`: data that supports easier searching and sorting of complaints submitted by or on behalf of consumers
* `consumer_consent_provided`: identifies whether the consumer option in to publish their complaint narrative
* `submitted_via`: how the complaint was submitted to the CFPB
* `date_sent_to_company`: date the CFBP sent the complaint to the company
* `company_response_to_consumer`: how the company responded
* `timely_response`: whether the company gave a timely response
* `consumer_disputed`: whether the company disputed the company's response
* `complaint_id`: unique identification number for a complaint

<a id='explore'></a>
# Explore

Through this notebook, we showed the power and simplicity of the **Teradata SQL Extension for Jupyter** and how it can be leveraged to extract insights from the data and help tell the story behind the dataset by answering questions. Hopefully you've noticed how easy it is to use **Vantage** to write your own SQL queries inside a Jupyter notebook. You may also use the Navigator found in the JupyterLab Launcher to explore the Vantage system catalog. For more information about the Navigator and additional **Teradata SQL Extension for Jupyter** functionality, refer to https://teradata.github.io/jupyterextensions/ (only available outside of the workspace).

You can continue to **EXPLORE Vantage** to extract more insights and find answers to other questions by leveraging the sandbox environment with the pre-loaded dataset. Here are some suggestions:

* **What are the most common types of complaints?** By grouping the `product` category, we can arrive at this answer. How does this change over time?
* **How are customers submitting their complaints?** The column `submitted_via` can also be grouped to answer for this question.
* **What proportion of the customer complaints are disputed?** By aggregating counts of `customer_disputed` we can answer this question.
* **Is there seasonality in the data? What is the reason for the seasonality?** If we subtract the trend from the series we can analyze the seasonality in the dataset. Are most of the complaints filed during the week or on the weekends?